In [1]:
rdd_review = sc.textFile("hdfs:///user/cloudera/data/input/sample_full.json")

In [2]:
import json

def get_prod_rating(review_json_item):
    dict_review_item = json.loads(review_json_item)
    return (dict_review_item["asin"], dict_review_item["overall"])

In [3]:
import csv, io

def list_to_csv_str(x):
    """Given a list of strings, returns a properly-csv-formatted string."""
    output = io.StringIO("")
    csv.writer(output).writerow(x)
    return output.getvalue().strip() # remove extra newline

In [4]:
rdd_prod_rating = rdd_review.map(lambda row: get_prod_rating(row))

In [5]:
rdd_avg_by_prod = rdd_prod_rating.aggregateByKey((0,0), 
                                        lambda x, value: (x[0] + value, x[1] + 1), 
                                        lambda x, y: (x[0] + y[0], x[1] + y[1])).mapValues(lambda x: x[0]/x[1])

In [ ]:
rdd_avg_by_prod.map(list_to_csv_str).saveAsTextFile("hdfs:///user/cloudera/data/output_spark_avg_rating")

In [6]:
rdd_avg_by_prod.map(list_to_csv_str).take(5)

['0594481813,4.0',
 '1400532655,3.8073394495412844',
 'B000001OM4,4.352941176470588',
 'B000001OMI,4.4',
 'B00000J3Q7,4.230769230769231']